In [24]:
import pandas as pd
import numpy as np
import torch

champions = ['Top', 'Jungle', 'Mid', 'Bot', 'Support', 'Enemy Top', 'Enemy Jungle', 'Enemy Mid', 'Enemy Bot', 'Enemy Support']
in_game_attributes = ['Team Gold', 'Team Kills', 'Team Deaths', 'Dragons', 'Dragon Soul', 'Baron', 'Rift Herald', 'First tower', 'First Kill', 'Team Total CC']


### 2-Step Model Testing

In [25]:
import pickle
with open('random_forest_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)
with open("scaler.pkl", "rb") as file:
    scaler = pickle.load(file)
with open("encoding_scheme.pkl", "rb") as file:
    encoding_scheme = pickle.load(file)

In [29]:
new_input = pd.DataFrame(columns=encoding_scheme)
row_with_zeroes = pd.Series(0, index=encoding_scheme)

# Add the row to the new_input DataFrame
new_input = pd.concat([new_input, row_with_zeroes.to_frame().T], ignore_index=True)
input_dict = {
    'Top': 'Singed',
    'Jungle': 'Taliyah',
    'Mid': 'Gangplank',
    'Bot': 'Zeri',
    'Support': 'Renata',
    'Enemy Top': 'Riven',
    'Enemy Jungle': 'Graves',
    'Enemy Mid': 'Swain',
    'Enemy Bot': 'Twitch',
    'Enemy Support': 'Lulu'
}
df = pd.DataFrame(input_dict, index=[0])
encoded_input = pd.get_dummies(df)
intersect = new_input.columns.intersection(encoded_input.columns)
new_input[intersect] = 1
display(new_input)

,Top_Aatrox,Top_Ahri,Top_Akali,Top_Akshan,Top_Alistar,Top_Amumu,Top_Anivia,Top_Annie,Top_Aphelios,Top_Ashe,...,Enemy Support_Yone,Enemy Support_Yorick,Enemy Support_Yuumi,Enemy Support_Zac,Enemy Support_Zed,Enemy Support_Zeri,Enemy Support_Ziggs,Enemy Support_Zilean,Enemy Support_Zoe,Enemy Support_Zyra
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
model = torch.load('fnn_num_model.pth')
model.eval()
input_tensor = torch.tensor(np.array(new_input, dtype=np.float32), dtype=torch.float32)

if torch.cuda.is_available():
    input_tensor = input_tensor.cuda()
with torch.no_grad():
    output = model(input_tensor)
    
post_out = np.round(scaler.inverse_transform(output.cpu().numpy())).astype(int)
out_table = pd.DataFrame(post_out, columns=in_game_attributes)
display(out_table)

,Team Gold,Team Kills,Team Deaths,Dragons,Dragon Soul,Baron,Rift Herald,First tower,First Kill,Team Total CC
0,47459,25,25,1,0,1,1,0,0,91


In [31]:
output = loaded_model.predict(post_out)
print(output[0])
prob_out = loaded_model.predict_proba(post_out)[:, 1]
print("{:.2f}%".format(prob_out[0] * 100))

False
38.90%
